In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt  # install

import torch
import utils
display = utils.notebook_init()  # checks

YOLOv5 🚀 v7.0-134-g23c4923 Python-3.9.16 torch-2.0.0+cu118 CUDA:0 (Tesla T4, 15102MiB)


Setup complete ✅ (2 CPUs, 12.7 GB RAM, 23.3/78.2 GB disk)


In [ ]:
!pip install ultralytics filterpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.1/510.1 KB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 KB 23.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 25.8 MB/s eta 0:00:00
  Created wheel for filterpy: filename=filterpy-1.4.5-py3-none-any.whl size=110473 sha256=b7575de202c73271ce7d71c5711beba68b03fef693a3d0c2bddc671fb6fa9509
  Stored in directory: /root/.cache/pip/wheels/53/e6/de/a09ea01e923aaf88b9f8c7c44329e857b2c1a31901167e55e6
Successfully built filterpy


In [ ]:
# YOLOv5 PyTorch HUB Inference (DetectionModels only)
from ultralytics import YOLO

model = YOLO("yolov8m.pt")
im = 'https://ultralytics.com/images/zidane.jpg'  # file, Path, PIL.Image, OpenCV, nparray, list
results = model(im)  # inference
# results  # or .show(), .save(), .crop(), .pandas(), etc.


Found https://ultralytics.com/images/zidane.jpg locally at zidane.jpg
image 1/1 /content/yolov5/zidane.jpg: 384x640 4 persons, 2 ties, 26.6ms
Speed: 0.4ms preprocess, 26.6ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 640)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import cv2
from ultralytics import YOLO
import numpy as np
from filterpy.kalman import KalmanFilter

model = YOLO("yolov8m.pt")

# cap = cv2.VideoCapture("street.mp4")
cap = cv2.VideoCapture("/content/drive/MyDrive/Colab Notebooks/Engenharia de Trafego/11h-12h.mp4")

width  = int(cap.get(3))  # float `width`
height = int(cap.get(4))


# Create a mask to cut the input
roi_corners = np.array([(0, 0), (1900, 0), (1900, 1900), (0, 1900)])
mask = np.zeros((height, width), dtype=np.uint8)
cv2.fillPoly(mask, [roi_corners], 255)

detection_classes = model("https://ultralytics.com/images/bus.jpg")[0].names
pass


100%|██████████| 476k/476k [00:00<00:00, 35.7MB/s]
image 1/1 /content/yolov5/bus.jpg: 640x480 4 persons, 1 bus, 115.6ms
Speed: 1.1ms preprocess, 115.6ms inference, 6.6ms postprocess per image at shape (1, 3, 640, 640)


In [ ]:
def linear_assignment(cost_matrix):
    try:
        import lap #linear assignment problem solver
        _, x, y = lap.lapjv(cost_matrix, extend_cost = True)
        return np.array([[y[i],i] for i in x if i>=0])
    except ImportError:
        from scipy.optimize import linear_sum_assignment
        x,y = linear_sum_assignment(cost_matrix)
        return np.array(list(zip(x,y)))
    
"""From SORT: Computes IOU between two boxes in the form [x1,y1,x2,y2]"""
def iou_batch(bb_test, bb_gt):
    
    bb_gt = np.expand_dims(bb_gt, 0)
    bb_test = np.expand_dims(bb_test, 1)
    
    xx1 = np.maximum(bb_test[...,0], bb_gt[..., 0])
    yy1 = np.maximum(bb_test[..., 1], bb_gt[..., 1])
    xx2 = np.minimum(bb_test[..., 2], bb_gt[..., 2])
    yy2 = np.minimum(bb_test[..., 3], bb_gt[..., 3])
    w = np.maximum(0., xx2 - xx1)
    h = np.maximum(0., yy2 - yy1)
    wh = w * h
    o = wh / ((bb_test[..., 2] - bb_test[..., 0]) * (bb_test[..., 3] - bb_test[..., 1])                                      
    + (bb_gt[..., 2] - bb_gt[..., 0]) * (bb_gt[..., 3] - bb_gt[..., 1]) - wh)
    return(o)

"""Takes a bounding box in the form [x1,y1,x2,y2] and returns z in the form [x,y,s,r] where x,y is the center of the box and s is the scale/area and r is the aspect ratio"""
def convert_bbox_to_z(bbox):
    w = bbox[2] - bbox[0]
    h = bbox[3] - bbox[1]
    x = bbox[0] + w/2.
    y = bbox[1] + h/2.
    s = w * h    
    #scale is just area
    r = w / float(h)
    return np.array([x, y, s, r]).reshape((4, 1))


"""Takes a bounding box in the centre form [x,y,s,r] and returns it in the form
    [x1,y1,x2,y2] where x1,y1 is the top left and x2,y2 is the bottom right"""
def convert_x_to_bbox(x, score=None):
    w = np.sqrt(x[2] * x[3])
    h = x[2] / w
    if(score==None):
        return np.array([x[0]-w/2.,x[1]-h/2.,x[0]+w/2.,x[1]+h/2.]).reshape((1,4))
    else:
        return np.array([x[0]-w/2.,x[1]-h/2.,x[0]+w/2.,x[1]+h/2.,score]).reshape((1,5))

from random import randint
def get_color():
    color = (randint(0, 255), randint(0, 255), randint(0, 255))
    return color

class KalmanBoxTracker(object):
    
    count = 0
    def __init__(self, bbox, initial_frame):
        """
        Initialize a tracker using initial bounding box
        
        Parameter 'bbox' must have 'detected class' int number at the -1 position.
        """
        self.kf = KalmanFilter(dim_x=7, dim_z=4)
        self.kf.F = np.array([[1,0,0,0,1,0,0],[0,1,0,0,0,1,0],[0,0,1,0,0,0,1],[0,0,0,1,0,0,0],[0,0,0,0,1,0,0],[0,0,0,0,0,1,0],[0,0,0,0,0,0,1]])
        self.kf.H = np.array([[1,0,0,0,0,0,0],[0,1,0,0,0,0,0],[0,0,1,0,0,0,0],[0,0,0,1,0,0,0]])

        self.kf.R[2:,2:] *= 10. # R: Covariance matrix of measurement noise (set to high for noisy inputs -> more 'inertia' of boxes')
        self.kf.P[4:,4:] *= 100. #give high uncertainty to the unobservable initial velocities
        self.kf.P *= 10.
        self.kf.Q[-1,-1] *= 0.5 # Q: Covariance matrix of process noise (set to high for erratically moving things)
        self.kf.Q[4:,4:] *= 0.5

        self.kf.x[:4] = convert_bbox_to_z(bbox) # STATE VECTOR
        self.time_since_update = 0
        self.id = KalmanBoxTracker.count
        KalmanBoxTracker.count += 1
        self.history = []
        self.hits = 0
        self.hit_streak = 0
        self.age = 0
        self.centroidarr = []
        CX = (bbox[0]+bbox[2])//2
        CY = (bbox[1]+bbox[3])//2
        self.centroidarr.append((CX,CY))
        
        
        #keep yolov5 detected class information
        self.detclass = bbox[5]
        self.detclass_name = detection_classes[self.detclass]
        self.color = get_color()
        self.length = bbox[2] - bbox[0]
        self.initial_frame = initial_frame
        
    def update(self, bbox):
        """
        Updates the state vector with observed bbox
        """
        self.time_since_update = 0
        self.history = []
        self.hits += 1
        self.hit_streak += 1
        self.kf.update(convert_bbox_to_z(bbox))
        self.detclass = bbox[5]
        CX = (bbox[0]+bbox[2])//2
        CY = (bbox[1]+bbox[3])//2
        self.centroidarr.append((CX,CY))
        
    def predict(self):
        """
        Advances the state vector and returns the predicted bounding box estimate
        """
        if((self.kf.x[6]+self.kf.x[2])<=0):
            self.kf.x[6] *= 0.0
        self.kf.predict()
        self.age += 1
        if(self.time_since_update>0):
            self.hit_streak = 0
        self.time_since_update += 1
        self.history.append(convert_x_to_bbox(self.kf.x))
        # bbox=self.history[-1]
        # CX = (bbox[0]+bbox[2])/2
        # CY = (bbox[1]+bbox[3])/2
        # self.centroidarr.append((CX,CY))
        
        return self.history[-1]
    
    
    def get_state(self):
        """
        Returns the current bounding box estimate
        # test
        arr1 = np.array([[1,2,3,4]])
        arr2 = np.array([0])
        arr3 = np.expand_dims(arr2, 0)
        np.concatenate((arr1,arr3), axis=1)
        """
        arr_detclass = np.expand_dims(np.array([self.detclass]), 0)
        
        arr_u_dot = np.expand_dims(self.kf.x[4],0)
        arr_v_dot = np.expand_dims(self.kf.x[5],0)
        arr_s_dot = np.expand_dims(self.kf.x[6],0)
        
        return np.concatenate((convert_x_to_bbox(self.kf.x), arr_detclass, arr_u_dot, arr_v_dot, arr_s_dot), axis=1)
    

def associate_detections_to_trackers(detections, trackers, iou_threshold = 0.3):
    """
    Assigns detections to tracked object (both represented as bounding boxes)
    Returns 3 lists of 
    1. matches,
    2. unmatched_detections
    3. unmatched_trackers
    """
    if(len(trackers)==0):
        return np.empty((0,2),dtype=int), np.arange(len(detections)), np.empty((0,5),dtype=int)
    
    iou_matrix = iou_batch(detections, trackers)
    
    if min(iou_matrix.shape) > 0:
        a = (iou_matrix > iou_threshold).astype(np.int32)
        if a.sum(1).max() == 1 and a.sum(0).max() ==1:
            matched_indices = np.stack(np.where(a), axis=1)
        else:
            matched_indices = linear_assignment(-iou_matrix)
    else:
        matched_indices = np.empty(shape=(0,2))
    
    unmatched_detections = []
    for d, det in enumerate(detections):
        if(d not in matched_indices[:,0]):
            unmatched_detections.append(d)
            
    
    unmatched_trackers = []
    for t, trk in enumerate(trackers):
        if(t not in matched_indices[:,1]):
            unmatched_trackers.append(t)
    
    #filter out matched with low IOU
    matches = []
    for m in matched_indices:
        if(iou_matrix[m[0], m[1]]<iou_threshold):
            unmatched_detections.append(m[0])
            unmatched_trackers.append(m[1])
        else:
            matches.append(m.reshape(1,2))
    
    if(len(matches)==0):
        matches = np.empty((0,2), dtype=int)
    else:
        matches = np.concatenate(matches, axis=0)
        
    return matches, np.array(unmatched_detections), np.array(unmatched_trackers)

class Sort(object):
    def __init__(self, max_age=1, min_hits=3, iou_threshold=0.3):
        """
        Parameters for SORT
        """
        self.max_age = max_age
        self.min_hits = min_hits
        self.iou_threshold = iou_threshold
        self.trackers = []
        self.frame_count = 0
        self.deleted_trackers = []
        

 
    def getTrackers(self,):
        return self.trackers
        
    def update(self, dets= np.empty((0,6)), frame_count=0):
        """
        Parameters:
        'dets' - a numpy array of detection in the format [[x1, y1, x2, y2, score], [x1,y1,x2,y2,score],...]
        
        Ensure to call this method even frame has no detections. (pass np.empty((0,5)))
        
        Returns a similar array, where the last column is object ID (replacing confidence score)
        
        NOTE: The number of objects returned may differ from the number of objects provided.
        """
        self.frame_count += 1
        
        # Get predicted locations from existing trackers
        trks = np.zeros((len(self.trackers), 6))
        to_del = []
        ret = []
        for t, trk in enumerate(trks):
           
            pos = self.trackers[t].predict()[0]
            trk[:] = [pos[0], pos[1], pos[2], pos[3], 0, 0]
            if np.any(np.isnan(pos)):
                to_del.append(t)
        trks = np.ma.compress_rows(np.ma.masked_invalid(trks))
        for t in reversed(to_del):
            self.deleted_trackers.append(self.trackers.pop(t))
        matched, unmatched_dets, unmatched_trks = associate_detections_to_trackers(dets, trks, self.iou_threshold)
        
        # Update matched trackers with assigned detections
        for m in matched:
            self.trackers[m[1]].update(dets[m[0], :])
            
        # Create and initialize new trackers for unmatched detections
        for i in unmatched_dets:
            trk = KalmanBoxTracker(np.hstack((dets[i,:], np.array([0]))), initial_frame = frame_count)
            self.trackers.append(trk)
        
        
        i = len(self.trackers)
        for trk in reversed(self.trackers):
            d = trk.get_state()[0]
            if (trk.time_since_update < 1) and (trk.hit_streak >= self.min_hits or self.frame_count <= self.min_hits):
                ret.append(np.concatenate((d, [trk.id+1])).reshape(1,-1)) #+1'd because MOT benchmark requires positive value
            i -= 1
            #remove dead tracklet
            if(trk.time_since_update >self.max_age):
                self.deleted_trackers.append(self.trackers.pop(i))
                

        if(len(ret) > 0):
            return np.concatenate(ret)
        return np.empty((0,6))

def crop_video_frame(frame, mask):
    frame = cv2.bitwise_and(frame, frame, mask=mask)
    # Crop the array using the minimum and maximum indices
    nonzero_indices = np.nonzero(frame)
    min_row = np.min(nonzero_indices[0])
    max_row = np.max(nonzero_indices[0])
    min_col = np.min(nonzero_indices[1])
    max_col = np.max(nonzero_indices[1])
    frame = frame[min_row:max_row+1, min_col:max_col+1]
    return frame




def draw_box(img, bbox, label, color):
    x1, y1, x2, y2 = [int(i) for i in bbox]
    tl = round(0.002 * (img.shape[0] + img.shape[1]) / 2) + 1  # line/font thickness

    cv2.rectangle(img, (x1, y1), (x2, y2), color, tl)

    tf = max(tl - 1, 1)  # font thickness
    t_size = cv2.getTextSize(label, 0, fontScale=tl / 3, thickness=tf)[0]
    c2 = x1 + t_size[0], y1 - t_size[1] - 3
    cv2.rectangle(img, (x1, y1), c2, color, -1, cv2.LINE_AA)  # filled
    cv2.putText(img, label, (x1, y1 - 2), 0, tl / 3, [225, 255, 255], thickness=tf, lineType=cv2.LINE_AA)
    return img

def extract_results(results):
    boxes = results[0].boxes.boxes

    target_classes = results[0].names
    labels = [target_classes[int(i)] for i in results[0].boxes.cls]
    return boxes, labels


In [ ]:
from tqdm import tqdm

In [ ]:
j = 0
target_classes = set([1, 2, 3, 5, 6, 7])

sort_tracker = Sort(max_age=15,
                       min_hits=0,
                       iou_threshold=0.1)

from shapely.geometry import Point, Polygon

corners = [(400, 285), (700, 250), (1000, 230), (1000, 380), (400, 500)]

# corners = [(0, 0), (100, 0), (100, 100), (0, 100)]

Roi = Polygon(corners)
mask = np.zeros((height, width), dtype=np.uint8)
cv2.fillPoly(mask, [np.array(corners)], 255)



vehicle_count_at_15_mim = []

while True:
    j += 1
    ret, frame = cap.read()
    
    if not ret:
        break
    
    # frame = crop_video_frame(frame, mask)

    result = model(frame, device='mps')[0]
    # frame = cv2.bitwise_and(frame, frame, mask=mask)
    # cv2.polylines(frame, [np.array(corners)], True, (0, 255, 0), 2)
    detections = result.boxes.boxes

    detecs = list(detections.cpu().numpy())
    detections = []
    for i, _ in enumerate(detecs):
        if Roi.contains(Point(_[2], _[3])):
            if _[-1] in target_classes and _[-2] > 0.7:
                detections.append(_)
    if len(detections) == 0:
        detections = np.empty((0,6))
    

    tracked_dets = sort_tracker.update(np.array(detections), frame_count=j)
    tracks = sort_tracker.getTrackers()

    # for track in tracks:
    #     track_color = track.color

    #     draw_box(frame, track.get_state()[:,:4].ravel(), str(track.id) + track.detclass_name, track_color)
        
    #     [cv2.line(frame, (int(track.centroidarr[i][0]),
    #                                             int(track.centroidarr[i][1])), 
    #                                             (int(track.centroidarr[i+1][0]),
    #                                             int(track.centroidarr[i+1][1])),
    #                                             track_color) 
    #                                             for i,_ in  enumerate(track.centroidarr) 
    #                                                 if i < len(track.centroidarr)-1 ]
     
    if j % (15*60*30 - 1) == 0:
        vehicle_count_at_15_mim.append(detections)
    
    # cv2.imshow("Img", frame)
    # key = cv2.waitKey(0)
    # if key == 27:
    #     break

Streaming output truncated to the last 5000 lines.
0: 384x640 1 person, 11 cars, 1 bus, 1 truck, 22.8ms
Speed: 0.4ms preprocess, 22.8ms inference, 1.9ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 1 person, 9 cars, 1 bus, 1 truck, 23.1ms
Speed: 0.5ms preprocess, 23.1ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 1 person, 10 cars, 1 bus, 2 trucks, 20.2ms
Speed: 0.4ms preprocess, 20.2ms inference, 5.5ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 9 cars, 1 bus, 1 truck, 23.2ms
Speed: 0.4ms preprocess, 23.2ms inference, 1.9ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 9 cars, 1 bus, 1 truck, 22.5ms
Speed: 0.4ms preprocess, 22.5ms inference, 1.7ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 10 cars, 1 bus, 1 truck, 25.6ms
Speed: 0.4ms preprocess, 25.6ms inference, 2.3ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 9 cars, 1 truck, 22.9ms
Speed: 0.4ms preprocess, 22.9ms inferenc

In [ ]:
import pandas as pd

trackers_objs = sort_tracker.trackers + sort_tracker.deleted_trackers

df = pd.DataFrame(columns=['id', 'type', 'initial_frame', 'detection_duration', 'lenght'])

for track in trackers_objs:
    df2 = pd.DataFrame({
        'id': track.id,
        'type': track.detclass_name,
        'initial_frame': track.initial_frame,
        'detection_duration': len(track.centroidarr),
        # 'detection_path': str(track.centroidarr),
        'lenght': track.length}, index=[0])
    
    df = pd.concat([df, df2], ignore_index=True)

df = df.sort_values(by=['id', 'initial_frame'])
df.to_csv('/content/drive/MyDrive/Colab Notebooks/Engenharia de Trafego/Cafu_results_full_11h_12h.csv', index=False) 
df  

,id,type,initial_frame,detection_duration,lenght
3,0,truck,1,24,211.272232
2,1,truck,11,2,196.519531
4,2,train,27,12,373.203491
5,3,truck,54,1,124.501465
6,4,train,55,4,324.723145
...,...,...,...,...,...
5620,5619,motorcycle,107847,9,58.390381
5619,5620,car,107860,1,46.302490
0,5621,car,107882,59,122.937012
5623,5622,car,107894,27,54.731567


In [ ]:
df.head()

,PUBLICO / NORMAL,PUBLICO / COM CONTRATO (PURA),RESIDENCIAL / VULNERAVEL NORMAL,INDUSTRIAL / NORMAL,COMERCIAL / NORMAL,COMERCIAL / ASSISTÊNCIA SOCIAL,RESIDENCIAL / NORMAL,PUBLICO / COM CONTRATO DE PROGRAMA,RESIDENCIAL / SOCIAL,Consumo(m3)
0,118.25,88.63,14.02,118.25,118.25,59.12,58.95,59.12,18.37,0
1,118.25,88.63,14.02,118.25,118.25,59.12,58.95,59.12,18.37,1
2,118.25,88.63,14.02,118.25,118.25,59.12,58.95,59.12,18.37,2
3,118.25,88.63,14.02,118.25,118.25,59.12,58.95,59.12,18.37,3
4,118.25,88.63,14.02,118.25,118.25,59.12,58.95,59.12,18.37,4


In [ ]:
import plotly.express as px
import pandas as pd

df = pd.read_csv('consumo_aguas.csv', index_col=0)
df['Consumo(m3)'] = df.index


In [ ]:
fig = px.line(df, x="Consumo(m3)", y=df.columns[:-1], title='Conta de agua por catogria')

fig.update_layout(
    yaxis_title="Valor da Conta de Água (R$)",
    legend_title="Categorias",
)

fig.show()
# fig.write_html("consumo_por_categoria.html")